__Chapter 14 - Going Deeper – The Mechanics of TensorFlow__

1. [TensorFlow ranks and tensors](#TensorFlow-ranks-and-tensors)
    1. [How to get the rank and shape of a tensor](#How-to-get-the-rank-and-shape-of-a-tensor)
1. [Understanding TensorFlow's computation graphs](#Understanding-TensorFlows-computation-graphs)
1. [Placeholders in TensorFlow](#Placeholders-in-TensorFlow)
    1. [Defining placeholders](#Defining-placeholders)
    1. [Feeding placeholders with data](#Feeding-placeholders-with-data)
    1. [Defining placeholders for data arrays with varying batchsizes](#Defining-placeholders-for-data-arrays-with-varying-batchsizes)
1. [Variables in TensorFlow](#Variables-in-TensorFlow)
    1. [Defining variables](#Defining-variables)
    1. [Initializing variables](#Initializing-variables)
    1. [Variable scope](#Variable-scope)
1. [](#)
1. [](#)
1. [](#)


In [1]:
# Standard libary and settings
import os
import sys
import importlib
import itertools
import warnings; warnings.simplefilter('ignore')
dataPath = os.path.abspath(os.path.join('../../Data'))
modulePath = os.path.abspath(os.path.join('../../CustomModules'))
sys.path.append(modulePath) if modulePath not in sys.path else None
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))


# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.6f}'.format


# Modeling extensions
import sklearn.base as base
import sklearn.cluster as cluster
import sklearn.datasets as datasets
import sklearn.decomposition as decomposition
import sklearn.discriminant_analysis as discriminant_analysis
import sklearn.ensemble as ensemble
import sklearn.feature_extraction as feature_extraction
import sklearn.feature_selection as feature_selection
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
import sklearn.neighbors as neighbors
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
import sklearn.svm as svm
import sklearn.tree as tree
import sklearn.utils as utils


# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt


# Custom extensions and settings
from quickplot import qp, qpUtil, qpStyle
from mlTools import powerGridSearch
sns.set(rc = qpStyle.rcGrey)


# Magic functions
%matplotlib inline


<a id = 'TensorFlow-ranks-and-tensors'></a>

# TensorFlow ranks and tensors

The TensorFlow library allows users to define graphs which perform operations and functions over tensors. Tensors are a generalizable mathematical for multidimensional arrays holding data values. The dimensionality of a tensor is generally referred to as its rank. Up to this point, we have worked with tensors of rank zero to two. A scalar is a tensor of rank 0, a vector is a tensor of rank 1, and a matrix is a tensor of rank 2. Tensor notation can be generalized to higher dimensions.

<a id = 'How-to-get-the-rank-and-shape-of-a-tensor'></a>

## How to get the rank and shape of a tensor

TensorFlow has several built in functions which easily facilitate retrieval of a Tensor's rank and shape.

In [2]:
import tensorflow as tf

g = tf.Graph()

# defint the computation graph
with g.as_default():
    # define tensors as t1, t2, t3
    t1 = tf.constant(np.pi)
    t2 = tf.constant([1,2,3,4])
    t3 = tf.constant([[1,2],[3,4]])
    
    # get the ranks
    r1 = tf.rank(t1)
    r2 = tf.rank(t2)
    r3 = tf.rank(t3)
    
    # get the shapes
    s1 = t1.get_shape()
    s2 = t2.get_shape()
    s3 = t3.get_shape()
    print('Shapes: {0}, {1}, {2}'.format(s1,s2,s3))
    
with tf.Session(graph = g) as sess:
    print('Ranks: {0}, {1}, {2}'.format(r1.eval(), r2.eval(), r3.eval()))

Shapes: (), (4,), (2, 2)
Ranks: 0, 1, 2


> Remarks - The rank of the t1 tensor is zero since it's just a scalar. The rank of the t2 vector is 1, and since it has four elements its shape is a one element tupe of (4,). The last shape t3 is a 2 by 2 matrix, and therfore has a rank of 2 and its shape is (2,2).

<a id = 'Understanding-TensorFlows-computation-graphs'></a>

## Understanding TensorFlow's computation graphs

At its core, TensorFlow relies on building a computation graph. This graph is used to derive relationships between tensors from the input all the way through to the output. As an example, if we have rank 0 tensor (a scalar) and tensors a, b and c, and we want to evaluate $z = 2 \times (a-b) + c$. The graph can be described as follows: Tensors a and b feed into $r_1$, which is equal to $a-b$. The resulting $r_1$ is referred to as an intermediate result tensor. $r_1 = a-b$ feeds into another intermediate graph $r_2$, which is equal to $2 \times r_1$. The intermediate result tensor $r_2$, along with c, are fed into the final result tensor $z$, which is equal to $r_2 + c$. This series of operations can be reimagined as a network of nodes, where each node represents an operation where a function is applied to one or more input tensors, and yields zero or more output tensors.

TensorFlow needs to be able to build this computational graph. The individual steps for building a typical graph are:

1. Instantiate a new, empty computation graph
2. Add nodes (tensors and operations) to the graph
3. Execute the graph by:
    1. Start a new session
    2. Initialize the variables in the graph
    3. Run the computation graph in this session

In [4]:
# computation graph for the equation above

g = tf.Graph()

with g.as_default():
    a = tf.constant(1, name = 'a')
    b = tf.constant(2, name = 'b')
    c = tf.constant(3, name = 'c')
    
    z = 2 * (a - b) + c


In the code above, 'with g.as_default()' adds nodes described within to the graph. By setting 'g' as the default graph, we are overriding the default graph that would otherwise takeover. Explicitly declaring which graph we want to use helps to avoid losing track of nodes.

A TensorFlow session is an enviroment in which operations and tensors of a graph can be executed. A session gets created by calling 'tf.Session' that receives an existing graph as an argument. If 'tf.Session(graph = g)' is called, the session uses the graph 'g', otherwise it uses the default graph, which may be empty and not what we expect.

Once a graph is launched in a TensorFlow session, we can execute its nodes containing tensors and/or its declared operations. Evaluating each individual tensor involves calling the 'eval' method inside the current session. When a specific tensor in the graph, TensorFlow executes all of the nodes that preceed that tensor until it reaches the tensor being specified.

Operations can also be excuted by using a session's 'run' method. In the previous chappter, an example executed an operator called 'train_op'. This operator doesn't return any tensor, but can still be executed by running 'train_op.run(). Further, there is a universal way of running both tensors and operators - 'tf.Session().run()'. This approach can be used to place multiple tensors and operators in a list or a tuple. As a result 'tf.Session().run()' will return a list or tuple of the same size.

Below, the previous graph is launched in a TensorFlow session and evaluates the tensor $z$ as follows:

In [6]:
# 

with tf.Session(graph = g) as sess:
    print('2 * (a - b) + c => {0}'.format(sess.run(z)))


2 * (a - b) + c => 1


<a id = 'Placeholders-in-TensorFlow'></a>

# Placeholders in TensorFlow

Placeholders are a special mechanism for feeding data into TensorFlow. These are predefined tensors with specific data types and shapes.

<a id = 'Defining placeholders'></a>

## Defining placeholders

Placeholders are defined using 'tf.placeholder'. The shape and data type are determined by the shape and the data type of the data that is fed into the placeholder at the time of execution. We can define the another graph that also evaluates $z = 2(a-b)+c$, but this time use placeholders for the scalars a, b and c. We also store the intermediate tensors $r_1$ and $r_2.

In [7]:
# 

g = tf.Graph()
with g.as_default():
    tf_a = tf.placeholder(tf.int32, shape = [], name = 'tf_a')
    tf_b = tf.placeholder(tf.int32, shape = [], name = 'tf_b')
    tf_c = tf.placeholder(tf.int32, shape = [], name = 'tf_c')
    
    r1 = tf_a - tf_b
    r2 = 2 * r1
    z = r2 + tf_c
    

> Remarks - Three placeholders, tf_a, tf_b and tf_c are set with the data type tf.int32 and a shape equal to [], which is what we use for rank 0 tensors (aka scalars). If we were using tensors of higher dimensions, such as a rank 3 tensor with a shape of 3 x 4 x 5, the shape might be [2, 3, 4]

<a id = 'Feeding-placeholders-with-data'></a>

## Feeding placeholders with data

A python dictionary is used to feed the values of placeholders into the session's graph. We need to ensure the data we feed in is consistent with the data type and shape of the placeholders. In the graph declared above, we have three placeholders that are tf.int32 scalars. Now we can feed in arbitrary int32 integers:

In [9]:
# 

with tf.Session(graph = g) as sess:
    feed = {tf_a : 1, tf_b : 2, tf_c : 3}
    print('z: {0}'.format(sess.run(z, feed_dict = feed)))
    

z: 1


<a id = 'Defining-placeholders-for-data-arrays-with-varying-batchsizes'></a>

## Defining placeholders for data arrays with varying batchsizes

Neural network models often deal with min-batches of data that have different sizes. For example, training may occur with mini-batches of a user-defined size, but then prediction may be made on only a single sample.

TensorFlow placeholders can accomodate this by specifying None for the dimension that will be varying in size.

In [12]:
# Create graph

g = tf.Graph()

with g.as_default():
    tf_x = tf.placeholder(tf.float32, shape = [None, 2], name = 'tf_x')
    x_mean = tf.reduce_mean(tf_x, axis = 0, name = 'mean')    

# Run graph in session
np.random.seed(123)

with tf.Session(graph = g) as sess:
    x1 = np.random.uniform(low = 0, high = 1, size = (5,2))
    print('Feeding data with shape: {}'.format(x1.shape))
    print('Result: {}'.format(sess.run(x_mean, feed_dict = {tf_x : x1})))
    
    x2 = np.random.uniform(low = 0, high = 1, size = (10,2))
    print('Feeding data with shape: {}'.format(x2.shape))
    print('Result: {}'.format(sess.run(x_mean, feed_dict = {tf_x : x2})))

Feeding data with shape: (5, 2)
Result: [0.6208972  0.46750155]
Feeding data with shape: (10, 2)
Result: [0.46306401 0.48766556]


In [14]:
# Review the shape of tf_x

print(tf_x)


Tensor("tf_x:0", shape=(?, 2), dtype=float32)


<a id = 'Variables-in-TensorFlow'></a>

# Variables in TensorFlow

Variables are a special type of tensor object that allows us to store and update parameters of our models in a TensoFlow session during the training process

<a id = 'Defining-variables'></a>

## Defining variables

TensorFlow variables store the parameters of a model that can be updated during training. The obvious example would the weights in the input, hidden and output layers of a neural network. When a variable is defined, we need to initialize it with a tensor of values. There are two ways to define variables

- tf.Variable()

This is a class that creates an object for a new variable and adds it to the graph. It does not have explicit shape and dtype parameters. These attributes are inferred from the initial values of the variable.

- tf.get_variable(

This approach enables us to reuse an existing variable with a certain name, or create a new variable if the name used does not exist in the graph yet. This approach also allows for declaration of dtype and shape, which are only necessary when declaring a new variable.

In both initialization techniques, the initial values are not set until the graph is launched via tf.Session.

In [18]:
# 

g1 = tf.Graph()

with g1.as_default():
    w = tf.Variable(np.array([[1, 2, 3, 4]
                             ,[5, 6, 7, 8]]), name = 'w')
    print(w)
    

<tf.Variable 'w:0' shape=(2, 4) dtype=int64_ref>


<a id = 'Initializing-variables'></a>

## Initializing variables

The computer memory that holds the variable is not allocated until the variables are initialized. Therefore, if we want to evaluate a tensor that includes a variable somewhere with its portion of the graph, we need to initialize the variables in that graph first. Initialization can occur using two methods. TensorFlow has a function named tf.global_variables_initializer, which, when executed, initializes the variables. We can also store this operator in an object, such as init_op = tf.global_variables_initializer(), which can then be executed later by sess.run(init_op) or init_op.run(). The key is that the operator is created after all variables that we need have been defined. A quick example illustrates this:

In [19]:
# 

g2 = tf.Graph()

with g2.as_default():
    w1 = tf.Variable(1, name = 'w1')
    init_op = tf.global_variables_initializer()
    w2 = tf.Variable(2, name = 'w2')


In [21]:
with tf.Session(graph = g2) as sess:
    sess.run(init_op)
    print('w1: {}'.format(sess.run(w1)))

w1: 1


In [22]:
with tf.Session(graph = g2) as sess:
    sess.run(init_op)
    print('w2: {}'.format(sess.run(w2)))

FailedPreconditionError: Attempting to use uninitialized value w2
	 [[Node: _retval_w2_0_0 = _Retval[T=DT_INT32, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](w2)]]

> Remarks - In the example above, 'w2' is declared after the variables are initialized, so it couldn't be evaluated. Reordering the lines would fix this.

<a id = 'Variable-scope'></a>

## Variable scope

Variable scopes allow variables to be organized into separate subparts. As an example, if we have two subnetworks within one neural network, we can define to scopres named 'net_A' and 'net_B'. Then each layer will be defined one of these two scopes.

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A